In [1]:
# File setup
from os.path import join
from os import chdir

BASE_PATH = "/workspaces/barco_skin_lesion_classification"
CODE_PATH = join(BASE_PATH, 'src/')

# Set the base path of the project
chdir(CODE_PATH)

In [2]:
# Imports
# Utils
import matplotlib as plt
import numpy as np
import wandb
import sys
import importlib
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import datetime


# DL libraries
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader

# User libraries
from datasets.segmentationdataset import SegmentationDataset
from models.unet_model import UNet
from trainers.segmentation_model_trainer import train_segmentation_model
from validators.segmentation_model_validator import validate_segmentation_model
from util import config

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [3]:
# Get the data
train_segmentation_dataset = SegmentationDataset(
    join(BASE_PATH, config.SEGMENTATION_DATA_PATH_TRAIN_FEATURES),
    join(BASE_PATH, config.SEGMENTATION_DATA_PATH_TRAIN_LABELS),
    config.SEGMENTATION_TRAIN_TRANSFORMATIONS_BOTH
    )

test_segmentation_dataset = SegmentationDataset(
    join(BASE_PATH, config.SEGMENTATION_DATA_PATH_TEST_FEATURES),
    join(BASE_PATH, config.SEGMENTATION_DATA_PATH_TEST_LABELS),
    config.SEGMENTATION_TEST_TRANSFORMATIONS_BOTH
    )

# Place the datasets in dataloaders
train_segmentation_dataloader = DataLoader(train_segmentation_dataset, batch_size=config.SEGMENTATION_BATCH_SIZE)
test_segmentation_dataloader = DataLoader(test_segmentation_dataset, batch_size=1)



# Setup

In [4]:
# Get the model
model = UNet(n_channels = 3, n_classes = 1)
model.to(config.DEVICE)

# Set the optimizer
optimizer = optim.Adam(model.parameters(), lr=config.SEGMENTATION_LR)

# Set the loss fn
criteria = nn.BCEWithLogitsLoss()

# Set the gradient scaler
grad_scaler = torch.cuda.amp.grad_scaler.GradScaler()


# Setup weights and biasses
wandb.login()

# Start wandb
wandb.init(
    settings=wandb.Settings(start_method="fork"),
    project="test-project", 
    name=f"experiment_{datetime.datetime.now()}", 
    config={
        "learning_rate": config.SEGMENTATION_LR,
        "batch_size": config.SEGMENTATION_BATCH_SIZE,
        "epochs": config.SEGMENTATION_EPOCHS,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: robberdg. Use `wandb login --relogin` to force relogin


# Training

In [5]:
# Set the variables to keep track of the best model
best_validation_loss = 10000
best_model_state = model.state_dict()

for epoch in range(config.SEGMENTATION_EPOCHS):
  # Set the model in training mode
  model.train()

  # Train the model
  total_train_loss_this_epoch = train_segmentation_model(
      model,
      optimizer,
      criteria,
      grad_scaler,
      train_segmentation_dataloader
  )
  
  # Set the model in evaluation mode
  model.eval()

  # Validate the model
  total_val_loss_this_epoch, sample_image_array = validate_segmentation_model(
      model,
      criteria,
      test_segmentation_dataloader,
      test_segmentation_dataset
  )

  # Convert the image array to a real imag object
  sample_image_array = sample_image_array.cpu()
  sample_image = Image.fromarray(np.uint8(sample_image_array) , 'L')

  # Calculate the loss values
  train_loss_this_epoch = total_train_loss_this_epoch/len(train_segmentation_dataloader.dataset)
  val_loss_this_epoch = total_val_loss_this_epoch/len(test_segmentation_dataloader.dataset)

  # Log the train loss this epoch
  wandb.log({
      'train_loss': train_loss_this_epoch,
      'val_loss': val_loss_this_epoch,
      'sample_image': wandb.Image(sample_image)
  })

  print(f'epoch: {epoch}, train_loss: {train_loss_this_epoch}, val_loss: {val_loss_this_epoch}')

  # If this is the best performing model yet, save it
  if val_loss_this_epoch < best_validation_loss:
    # Update the best value
    best_validation_loss = val_loss_this_epoch

    # Update the best model
    best_model_state = model.state_dict()

    # Save the best model
    checkpoint_path = join(
      BASE_PATH, 
      config.SEGMENTATION_MODEL_CHECKPOINT_PATH, 
      f'chechpoint_{datetime.datetime.now()}.pth'
    )
    torch.save(best_model_state, checkpoint_path)

100%|██████████| 100/100 [00:10<00:00,  9.43it/s]


epoch: 0, train_loss: 0.05720336062485634, val_loss: 1.7504621744155884


100%|██████████| 100/100 [00:10<00:00,  9.43it/s]


epoch: 1, train_loss: 0.048542023469853035, val_loss: 0.9512575268745422


100%|██████████| 100/100 [00:10<00:00,  9.32it/s]


...

100%|██████████| 100/100 [00:10<00:00,  9.42it/s]


epoch: 43, train_loss: 0.009962695503626811, val_loss: 0.5910130143165588


100%|██████████| 100/100 [00:10<00:00,  9.46it/s]


epoch: 44, train_loss: 0.0095821315633266, val_loss: 0.5358220338821411


 47%|████▋     | 47/100 [00:06<00:01, 27.07it/s]

In [ ]:
# Save the best model
checkpoint_path = join(
    BASE_PATH, 
    config.SEGMENTATION_MODEL_CHECKPOINT_PATH, 
    f'chechpoint_{datetime.datetime.now()}.pth'
  )
torch.save(best_model_state, checkpoint_path)


In [ ]:
# Mark the run as finished
wandb.finish()